#### Configurações iniciais

In [0]:
# Importar as bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Iniciar a SparkSession com configurações otimizadas
spark = SparkSession.builder \
    .appName("Load Data Bronze") \
    .config("spark.sql.shuffle.partitions", "200")  \
    .config("spark.sql.files.maxPartitionBytes", "128MB") \
    .config("spark.sql.parquet.compression.codec", "snappy") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

# Definir caminhos de armazenamento no Data Lake
lz_path_in = "/FileStore/lhdw/landingzone/vendas/processar"
lz_path_out = "/FileStore/lhdw/landingzone/vendas/processado"
bronze_path = "/FileStore/lhdw/bronze/vendas"


In [0]:
#dbutils.fs.cp('/FileStore/lhdw/landingzone/vendas/processado/dados_2012.csv',lz_path_in)
#dbutils.fs.rm('/FileStore/lhdw/landingzone/vendas/processado/dados_2012.csv')

####Criando schema dos dados

In [0]:
schema_lz = StructType([
    StructField("IDProduto", IntegerType(), True),
    StructField("Data", DateType(), True),
    StructField("IDCliente", IntegerType(), True),
    StructField("IDCampanha", IntegerType(), True),
    StructField("Unidades", IntegerType(), True),
    StructField("Produto", StringType(), True),
    StructField("Categoria", StringType(), True),
    StructField("Segmento", StringType(), True),
    StructField("IDFabricante", IntegerType(), True),
    StructField("Fabricante", StringType(), True),
    StructField("CustoUnitario", DoubleType(), True),
    StructField("PrecoUnitario", DoubleType(), True),
    StructField("CodigoPostal", StringType(), True),
    StructField("EmailNome", StringType(), True),
    StructField("Cidade", StringType(), True),
    StructField("Estado", StringType(), True),
    StructField("Regiao", StringType(), True),
    StructField("Distrito", StringType(), True),
    StructField("Pais", StringType(), True)
])

####Importando arquivo csv e Salvando na camada BRONZE

In [0]:
# Lista o conteúdo da pasta
arquivos = dbutils.fs.ls(lz_path_in)

for arquivo in arquivos: 
    #Importando o csv e adicionando uma coluna com o nome do arquivo importado.
    df_vendas = spark.read.option("header","true").schema(schema_lz).csv(f'{lz_path_in}/{arquivo.name}') \
        .withColumn("filename", regexp_extract(input_file_name(), "([^/]+)$", 0))

    #Adicionar colunas de Ano e Mes
    df_vendas = df_vendas.withColumn("Ano",year("Data"))\
        .withColumn("Mes",month("Data"))

    #Separando nome dos arquivos processados para serem movidos para o diretorio processado.
    file_name = df_vendas.select('filename').distinct()
    
    #Exportar arquivo em formato Parquet particionado por Ano e Mes
    df_vendas.write.mode("overwrite").partitionBy("Ano","Mes").parquet(f'{bronze_path}/{arquivo.name}')

    #Cria um CSV com os arquivos que foram processados para ser utilizados na camada Silver.
    file_name.write.mode("append").csv(f'{bronze_path}/files_to_silver_process',header=True)

    #Move o arquivo processado para o diretório de Processados.
    dbutils.fs.cp(f'{lz_path_in}/{arquivo.name}', lz_path_out)
    dbutils.fs.rm(f'{lz_path_in}/{arquivo.name}')


    


####Deletando caches

In [0]:
import gc
gc.collect()